<a href="https://colab.research.google.com/github/yash1ma/2019_kitajima/blob/master/L0_L1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install sporco
! pip install keras

     |████████████████████████████████| 4.4MB 2.7MB/s 
     |████████████████████████████████| 2.6MB 19.7MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import time
import random
import scipy.io as sio

import sys
sys.path.append('/content/drive/My Drive/研究/専攻科研究/2020_kitajima')
import Convert as con
import ConvDL as cdl
import conduct_svm as svm
import conduct_pooling as cp
import output as op
import load_dataset as ds
import random

In [ ]:
S = np.random.randn(32, 32, 10)
D = np.random.randn(12, 12, 5)
cri = cnvrep.CSC_ConvRepIndexing(D, S)
print(cri)

In [4]:
def make_option():
    opt = {}
    # 最大繰り返し回数
    opt['iteration'] = {"xstep": 10000, "dstep": 10000, "total": 10}
    # #学習の層の数
    # opt['Learning'] = 2
    # フィルターのサイズと数
    opt['cropped_shpD'] = (8, 8, 9)
    opt['nnCoef'] = True
    # opt['cropped_shpD'] = ((6, 6, 3), (9, 9, 3), (12, 12, 3))
    #パラメータ設定
    opt['lmbda'] = 1e-03
    opt['rho_c'] =  50.0*opt['lmbda'] + 0.5
    opt['rho_d'] = 1.0
    opt["threshold"] = {"xstep": 1e-05, "dstep": 1e-05, "tol_x": 1e-05, "tol_d": 1e-05}

    # #実験結果の保存先
    # opt['file_path'] = 'C:\\CDL\\result'
    return opt

np.random.seed(1)
train_amount = 48

# train_data, train_label = ds.load_fashion_train(train_amount)
# test_data, test_label = ds.load_fashion_test(train_amount)
# train_data = train_data.transpose(1, 2, 0)
# test_data = test_data.transpose(1, 2, 0)

mat_contents = sio.loadmat("/content/drive/My Drive/研究/専攻科研究/YaleB_32x32.mat", appendmat=True)
imgs = mat_contents.get("fea")[:10*64]
imgs = np.rot90(imgs.T.reshape(32, 32, 10, 64), -1, axes=(0,1))/255.0
labels = mat_contents.get("gnd")[:10*64].reshape(10, 64)
idx = np.random.choice(64, 64, replace=False)
train_data = imgs[0:, 0:, 0:, idx[:train_amount]]
test_data = imgs[0:, 0:, 0:, idx[train_amount:]]
train_label = labels[0:, idx[:train_amount]]
test_label = labels[0:, idx[train_amount:]]

辞書学習

In [5]:
D0 = np.random.randn(8, 8, 9)
opt = make_option()

S0 = train_data.reshape(train_data.shape[0], train_data.shape[1], -1)
# S0 = train_data
D1, X1, L0_1 = cdl.coef_dic_update_L1_L1(S0, D0, opt, 'Layer1')

0.11057920102719908
221.28563146840568
0.11057920102719908
4823.586917464105
0.11049669053819444
1251.2496658249313
0.11049601236979166
1742.997658368576
0.11042050962094907
1935.3091148527344
0.11049442997685185
2841.136464016024
0.11061604817708333
3780.2920201194756
0.11127658420138889
6164.5468009886135
0.11106635199652778
5629.065269524319
0.11123792860243055
3788.3772514520992



In [2]:
D1, X1, L0_1 = cdl.coef_dic_update_L1_L1(S0, D1, opt, 'Layer1')

NameError: ignored

In [1]:
from sporco import plot, util
plot.config_notebook_plotting()
plot.figure()
plot.imview(util.tiledict(D0))
plot.imview(util.tiledict(D1))

ModuleNotFoundError: ignored

In [ ]:
# 係数を特徴ベクトルに変形
train_feature = train_X1.reshape(train_X1.shape[3], -1)
test_feature = test_X1.reshape(test_X1.shape[3], -1)

# # 特徴ベクトルを次元削減
# if train_amount*10 < 256:
#     nmf = NMF(n_components = train_amount)
# else:
#     nmf = NMF(n_components = 256)

# nmf.fit(train_feature)

# train_feature = nmf.transform(train_feature)
# test_feature = nmf.transform(test_feature)

識別

In [ ]:
from scipy.optimize import nnls
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
from sklearn import svm

# NMFによる錐制約部分空間法
def create_cone_NMF(feature, num):
    nmf = NMF(n_components=num);
    W = nmf.fit_transform(feature);
    H = nmf.components_;
    return nmf, H, W;

def restore_from_cone_NMF(feature, nmf):
    restore = nmf.inverse_transform(nmf.transform(feature));
    return restore;

def NMF_Classifier(train_feature, train_label, test_feature, test_label, from_, to_):
    forcsv = []
    forcsv.append(["EXPERIMENT", "NMF CONE"]);
    forcsv.append(["dim", "accuracy"]);
    for h in range(from_, to_+1):
        NMFs = [];
        Hs = [];
        Ws = [];
        for i in range(10):
            #print(str(i)+"番目のクラスの錐を作成中...");
            nmf, H, W = create_cone_NMF(train_feature[train_label == i], h);
            NMFs.append(nmf);
            Hs.append(H);
            Ws.append(W);
        restores = [];
        for i in range(10):
            #print(str(i)+"番目のクラスの錐で入力を再現中...")
            temp = restore_from_cone_NMF(test_feature, NMFs[i]);
            restores.append(temp);
        CosArray = [];
        for i in range(10):
            cos_array = cosine_similarity(test_feature, restores[i]);
            CosArray.append(np.diag(cos_array));            
        CosArray = np.array(CosArray);
        predict_label = np.argmax(CosArray, axis = 0);
        precision = sum(predict_label == test_label)/test_label.shape[0];
        forcsv.append([h, precision])
        print("d_num="+str(h)+": Test precision: ", precision);
    return forcsv;

d_num = 128
result_nmf = NMF_Classifier(train_feature, train_labels.reshape(-1), test_feature, test_labels.reshape(-1), 1, d_num)

d_num=1: Test precision:  0.175
d_num=2: Test precision:  0.16875
d_num=3: Test precision:  0.1875
d_num=4: Test precision:  0.19375
d_num=5: Test precision:  0.21875
d_num=6: Test precision:  0.2125
d_num=7: Test precision:  0.225
d_num=8: Test precision:  0.2375
d_num=9: Test precision:  0.23125
d_num=10: Test precision:  0.24375
d_num=11: Test precision:  0.25625
d_num=12: Test precision:  0.24375
d_num=13: Test precision:  0.24375
d_num=14: Test precision:  0.2375
d_num=15: Test precision:  0.24375
d_num=16: Test precision:  0.25
d_num=17: Test precision:  0.2375
d_num=18: Test precision:  0.26875
d_num=19: Test precision:  0.25
d_num=20: Test precision:  0.24375
d_num=21: Test precision:  0.2375
d_num=22: Test precision:  0.25625
d_num=23: Test precision:  0.2375
d_num=24: Test precision:  0.23125
d_num=25: Test precision:  0.225
d_num=26: Test precision:  0.23125
d_num=27: Test precision:  0.2375
d_num=28: Test precision:  0.2375
d_num=29: Test precision:  0.2375
d_num=30: Test p

KeyboardInterrupt: ignored

In [ ]:
# 包括凸錐による錐制約部分空間法
def create_cone_PCA(feature, param, ratio):
    feature = normalize(feature);
    pca = PCA(n_components=ratio);
    pca.fit(feature);
    variance = pca.explained_variance_;
    mean = pca.mean_;
    basis = pca.components_;
    cone_basis = [];
    cnt = 0;
    for i in variance:
        cone_basis.append(mean+param*np.sqrt(i)*basis[cnt]);
        cone_basis.append(mean-param*np.sqrt(i)*basis[cnt]);
        cnt=cnt+1;
    cone_basis = np.array(cone_basis);
    return cone_basis;

def restore_from_cone_PCA(cone_basis, feature):
    feature = list(feature);
    restore = [];
    for i in feature:
        x = np.array(nnls(cone_basis. T, i)[0]);
        restore.append(cone_basis.T.dot(x));
    restore = np.array(restore);
    return restore;

def PCA_Classifier(train_feature, train_label, test_feature, test_label, params, ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999, 0.9999]):
    forcsv=[]
    forcsv.append(["EXPERIMENT", "COMPREHENSIVE CONE"])
    for param in params:
        forcsv.append(["scaling param", str(param)])
        forcsv.append(["contribution rate", "accuracy"])
        print("the parameter of scaling : "+str(param))
        for ratio in ratios:
            Cones = [];

            for i in range(10):
                #print(str(i)+"番目のクラスの錐を作成中...");
                cone = create_cone_PCA(train_feature[train_label == i], param, ratio);
                Cones.append(cone);
            restores = [];
            for i in range(10):
                #print(str(i)+"番目のクラスの錐で入力を再現中...")
                temp = restore_from_cone_PCA(Cones[i], test_feature);
                restores.append(temp);
            CosArray = [];
            for i in range(10):
                cos_array = cosine_similarity(test_feature, restores[i]);
                CosArray.append(np.diag(cos_array));

            CosArray = np.array(CosArray);
            predict_label = np.argmax(CosArray, axis = 0);
            precision = sum(predict_label == test_label)/test_label.shape[0];
            print("ratio="+str(ratio)+": Test precision: ", precision);
            forcsv.append([ratio, precision])
    return forcsv;

# PCAによる包括凸錐による部分空間法における累積寄与率のリスト・パラメータ
ratios = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999];
params = [2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0];
result_pca = PCA_Classifier(train_feature, train_labels.reshape(-1), test_feature, test_labels.reshape(-1), params, ratios)

the parameter of scaling : 2.5
ratio=0.3: Test precision:  0.2625
ratio=0.4: Test precision:  0.28125
ratio=0.5: Test precision:  0.26875
ratio=0.6: Test precision:  0.275
ratio=0.7: Test precision:  0.275
ratio=0.8: Test precision:  0.275
ratio=0.9: Test precision:  0.2625
ratio=0.99: Test precision:  0.275
ratio=0.999: Test precision:  0.275
the parameter of scaling : 3.0
ratio=0.3: Test precision:  0.25625
ratio=0.4: Test precision:  0.28125
ratio=0.5: Test precision:  0.2875
ratio=0.6: Test precision:  0.2875
ratio=0.7: Test precision:  0.26875
ratio=0.8: Test precision:  0.275
ratio=0.9: Test precision:  0.25625
ratio=0.99: Test precision:  0.26875
ratio=0.999: Test precision:  0.275
the parameter of scaling : 3.5
ratio=0.3: Test precision:  0.25625
ratio=0.4: Test precision:  0.28125
ratio=0.5: Test precision:  0.275
ratio=0.6: Test precision:  0.2875
ratio=0.7: Test precision:  0.28125
ratio=0.8: Test precision:  0.275
ratio=0.9: Test precision:  0.2625
ratio=0.99: Test precisio

KeyboardInterrupt: ignored

In [ ]:
# サポートベクターマシン
clf = svm.SVC(gamma='auto',kernel='linear')
print(clf.fit(train_feature, train_labels.reshape(-1)))
print(clf.score(test_feature, test_labels.reshape(-1)))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.2125


In [ ]:
from tqdm.notebook import tqdm
import time
pbar = tqdm(range(4))
for char in pbar:
    time.sleep(0.25)
    pbar.set_description("Processing %s" % char)

In [9]:
import matplotlib.pyplot as plt
plt.imshow(train_data[0]*255)

TypeError: ignored

学習器の訓練

In [ ]:
# 訓練用画像の係数算出
F1_train, L0_1_train = cdl.feature_extraction_L1_L0(rho_coef, gamma, mu, train_data, D1, iteration, thr, 'Feature1')